<a href="https://colab.research.google.com/github/DinisMartinho/YOLOv5s-Wildfire-Detection/blob/main/Wildfire%20%26%20Smoke%20Detection%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5 #      Clone the YOLOv5 repository.
!pip install -U -r yolov5/requirements.txt #            Install the necessary requirements.
import torch, IPython.display, yaml #                   Import the necessary libraries.
IPython.display.clear_output()

In [ ]:
# Download the "Wildfire Smoke Dataset" from the Roboflow website. ("https://public.roboflow.com/object-detection/wildfire-smoke")
!curl -L "PASTE YOUR ROBOFLOW DOWNLOAD LINK HERE" > roboflow.zip; unzip roboflow; rm roboflow.zip

In [ ]:
# Customize the IPython writefile function so that we can write variables.
@IPython.core.magic.register_line_cell_magic
def writetemplate(line, cell):
  with open(line, "w") as f:
    f.write(cell.format(**globals()))

In [ ]:
# Define the number of classes based on the YAML file.
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
# Choose in what device we should run this notebook on.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Rewrite YOLOs configuration file.
%%writetemplate /content/yolov5/models/custom_yolo5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# Anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 Backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 Head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# Train your YOLOv5s model.
%%time
%cd /content/yolov5/
!python train.py --img 737 --batch 16 --epochs 200 --data '../data.yaml' --cfg ./models/custom_yolo5s.yaml --weights '' --name yolov5s_results  --cache

In [ ]:
# You can find your trained weights in this directory.
%ls runs/

In [ ]:
# Inference test your trained model.
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.7 --source ../test/images

In [ ]:
# You can save your trained weights by copying the weights file into your google drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp runs/train/yolov5s_results/weights/best.pt ../drive/MyDrive

In [ ]:
!nvidia-smi